In [ ]:
#required imports
import numpy as np, matplotlib.pyplot as plt
import sep, astropy.io.fits as fits
from matplotlib import rcParams
from matplotlib.patches import Ellipse

%matplotlib inline

In [ ]:
#import fits formatted data to array
f105data = fits.getdata("./data/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
f105data = f105data.byteswap().newbyteorder()

#data analysis
f105mean, f105std = np.mean(f105data), np.std(f105data)

#background removal processing
#I'll come back to find out more about the parameters in use here
f105bkg = sep.Background(f105data, bw=64, bh=64, fw=3, fh=3)
f105bkg_image = f105bkg.back()
f105bkg_rms = f105bkg.rms()
f105data_sub = f105data - f105bkg

#get the objects
f105objects = sep.extract(f105data_sub, 1.5, err=f105bkg.globalrms)

In [ ]:
#plot f105 background-subtracted image
f, ax = plt.subplots()
f105sub_mean, f105sub_std = np.mean(f105data_sub), np.std(f105data_sub)
im = ax.imshow(f105data_sub, interpolation = 'nearest', cmap = 'gray',
               vmin = f105sub_mean - f105sub_std, vmax = f105sub_mean + f105sub_std, origin='lower')

#save a clean copy
f.set_facecolor("white")
f.savefig("./output/f105 clean.png", bbox_inches = "tight", dpi = 300)

#plot an ellipse for each object
for i in range(len(f105objects)):
    f105e = Ellipse(xy = (f105objects['x'][i], f105objects['y'][i]),
                width = 6 * f105objects['a'][i],
                height = 6 * f105objects['b'][i],
                angle = f105objects['theta'][i] * 180. / np.pi)
    f105e.set_facecolor('none')
    f105e.set_edgecolor('red')
    ax.add_artist(f105e)
    
f.savefig("./output/f105 identification.png", bbox_inches = "tight", dpi = 300)

In [ ]:
#calculate the fluxes
f105flux, f105fluxerr, f105flag = sep.sum_circle(f105data_sub, f105objects['x'], f105objects['y'],
                                     3.0, err = f105bkg.globalrms, gain = 1.0)

#plot on a histogram
f, ax = plt.subplots()
ax.hist(f105flux, 50)
ax.set_xlabel("Flux")
ax.set_ylabel("Occurrences")

#save histogram of raw data
f.set_facecolor("white")
f.savefig("./output/f105 raw flux histogram.png", bbox_inches = "tight", dpi = 300)

In [ ]:
#init a temp array to hold edges
hold  = []
#find literal edge cases
for i in range(len(f105flux)):
    if f105flux[i] < 3:
        hold.append(f105flux[i])

#remove edge cases
edges = np.array(hold)
f105fluxclean = np.setdiff1d(f105flux, edges)

#print the number of remaining sources
print(f"The cleaned number of sources is: {len(f105fluxclean)}")

In [ ]:
#replot with attempt at removing IDs of noise on the edge
#the point is to be able to see the values not in the first box
#using a log scale for the y-axis would be better, but I'm not sure how to at the moment.
f, ax = plt.subplots()
ax.hist(f105fluxclean, 50)
ax.set_xlabel("Flux")
ax.set_ylabel("Occurrences")

#save histogram of cleaned data
f.set_facecolor("white")
f.savefig("./output/f105 clean flux histogram.png", bbox_inches = "tight", dpi = 300)

In [ ]:
#calculate stats
f105mean = np.mean(f105flux)
f105median = np.median(f105flux)
f105std = np.std(f105flux)

print(f"The mean is: {f105mean:.2f}")
print(f"The median is: {f105median:.2f}")
print(f"The std is: {f105std:.2f}")

f105fluxmax = f105flux.max()

print(f"The largest outlier is {f105fluxmax:.2f}.")

temp = 0
f105fluxmaxloc = np.argmax(f105flux)
        
f105fluxmaxobj = f105objects[f105fluxmaxloc]

print(f"It is located from {f105fluxmaxobj[3]} to {f105fluxmaxobj[4]} on the x axis and {f105fluxmaxobj[5]} to {f105fluxmaxobj[6]} on the y axis.")

f105fluxmaxsigma = (f105fluxmax - f105mean) / f105std

print(f"It was a(n) {f105fluxmaxsigma:.2f}-sigma event.")

In [ ]:
#import fits formatted data to array
f125data = fits.getdata("./data/hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")
f125data = f125data.byteswap().newbyteorder()
f160data = fits.getdata("./data/hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")
f160data = f160data.byteswap().newbyteorder()

In [ ]:
#background removal of f125
f125mean, f125std = np.mean(f125data), np.std(f125data)

#background removal processing
f125bkg = sep.Background(f125data, bw=64, bh=64, fw=3, fh=3)
f125bkg_image = f125bkg.back()
f125bkg_rms = f125bkg.rms()
f125data_sub = f125data - f125bkg

#background removal of f160
f160mean, f160std = np.mean(f160data), np.std(f160data)

#background removal processing
#I'll come back to find out more about the parameters in use here
f160bkg = sep.Background(f160data, bw=64, bh=64, fw=3, fh=3)
f160bkg_image = f160bkg.back()
f160bkg_rms = f160bkg.rms()
f160data_sub = f160data - f160bkg

In [ ]:
'''
the plan is: determine the structure of each f***data_sub
if necessary, sum to a single dimension
append them to an array of the form: "(M, N, 3): an image with RGB values (0-1 float or 0-255 int)."
where M is the x-value, and each of the 3 Ns correspond to R, G, B
then pass result to .imshow
'''

In [ ]:
#plot the image on the different channels
f, ax = plt.subplots()

#f160 on red channel
f160sub_mean, f160sub_std = np.mean(f160data_sub), np.std(f160data_sub)

im = ax.imshow(f160data_sub, interpolation = 'nearest', cmap = 'Reds',
               vmin = f160sub_mean - f160sub_std, vmax = f160sub_mean + f160sub_std, origin='lower')

#f125 on green channel
f125sub_mean, f125sub_std = np.mean(f125data_sub), np.std(f125data_sub)

im = ax.imshow(f125data_sub, interpolation = 'nearest', cmap = 'Greens',
               vmin = f125sub_mean - f125sub_std, vmax = f125sub_mean + f125sub_std, origin='lower')

#f105 on blue channel
f105sub_mean, f105sub_std = np.mean(f105data_sub), np.std(f105data_sub)

im = ax.imshow(f105data_sub, interpolation = 'nearest', cmap = 'Blues',
               vmin = f105sub_mean - f105sub_std, vmax = f105sub_mean + f105sub_std, origin='lower')

#save image with false color
f.set_facecolor("white")
f.savefig("./output/composite false color.png", bbox_inches = "tight", dpi = 300)